In [1]:
import pandas
import sys
import os
import json
sys.path.insert(0, os.getcwd())
from helpers import recursive_find, read_json, create_fedora_results_table

In [2]:
# Read in the experiment json files. One experiment means one package at once version
# across splices (versions of a single dependencny) and predictors
experiments = list(recursive_find("artifacts/results/extracted/fedora", "*.json"))
print('Found %s experiment library files.' % len(experiments))

Found 7212 experiment library files.


In [3]:
df = create_fedora_results_table(experiments)
display(df)

,a,b,original,changed,analysis,seconds,predictor,prediction
0,fedora-libs-35,fedora-libs-37,first/usr/lib64/libQt5Quick.so.5.15.5,second/usr/lib64/libQt5Quick.so.5.15.5,missing-previously-found-symbols,0.002819,symbols,True
1,fedora-libs-35,fedora-libs-37,first/usr/lib64/libQt5Quick.so.5.15.5,second/usr/lib64/libQt5Quick.so.5.15.5,missing-previously-found-exports,0.254755,symbols,True
2,fedora-libs-35,fedora-libs-37,first/usr/lib64/libQt5Quick.so.5.15.5,second/usr/lib64/libQt5Quick.so.5.15.5,abidiff,49.429475,libabigail,True
3,fedora-libs-35,fedora-libs-37,first/usr/lib64/libQt5Quick.so.5.15.5,second/usr/lib64/libQt5Quick.so.5.15.5,abi-compliance-tester,625.529167,abi-laboratory,True
4,fedora-libs-34,fedora-libs-37,first/usr/lib64/libx11globalcomm.so.1.0.0,second/usr/lib64/libx11globalcomm.so.1.0.0,missing-previously-found-symbols,0.000072,symbols,True
...,...,...,...,...,...,...,...,...
28843,fedora-libs-34,fedora-libs-36,first/usr/lib/systemd/libsystemd-shared-250.so,second/usr/lib/systemd/libsystemd-shared-250.so,abi-compliance-tester,31.900113,abi-laboratory,True
28844,fedora-libs-36,fedora-libs-37,first/usr/lib/systemd/libsystemd-shared-250.so,second/usr/lib/systemd/libsystemd-shared-250.so,missing-previously-found-symbols,0.000897,symbols,True
28845,fedora-libs-36,fedora-libs-37,first/usr/lib/systemd/libsystemd-shared-250.so,second/usr/lib/systemd/libsystemd-shared-250.so,missing-previously-found-exports,0.106127,symbols,True
28846,fedora-libs-36,fedora-libs-37,first/usr/lib/systemd/libsystemd-shared-250.so,second/usr/lib/systemd/libsystemd-shared-250.so,abidiff,1.399777,libabigail,True


In [4]:
# How many unique libraries?
library_count = list(df['original'].values) + list(df['changed'].values)
print('Found %s total libraries' % len(library_count))

Found 57696 total libraries


In [5]:
# Symbols has two tests - we are using missing-previously-found-symbols
#df = df[df["analysis"] != "missing-previously-found-exports"]

# Remove unknown predictions
df = df[df['prediction'] != "Unknown"]

# Look to see if we have True/False
for predictor in df['predictor'].unique():
    print(predictor)
    print(df[ df['predictor']==predictor]['prediction'].unique())

# disagree = df.groupby(["a","b", "original", "changed"])['prediction'].nunique() > 1
# disagree = pandas.DataFrame(disagree)
# Where all predictors agree
all_predictors_agree = pandas.DataFrame(df.groupby(['a', 'b', 'original', 'changed'])['prediction'].nunique() == 1).reset_index()
df_all_agree = df.merge(all_predictors_agree, on=['a', 'b', 'original', 'changed'], suffixes=["", "_all_agree"])
df_all_agree = df_all_agree[df_all_agree['prediction_all_agree']]
df_all_agree

symbols
[True]
libabigail
[True False]
abi-laboratory
[True False]


,a,b,original,changed,analysis,seconds,predictor,prediction,prediction_all_agree
0,fedora-libs-35,fedora-libs-37,first/usr/lib64/libQt5Quick.so.5.15.5,second/usr/lib64/libQt5Quick.so.5.15.5,missing-previously-found-symbols,0.002819,symbols,True,True
1,fedora-libs-35,fedora-libs-37,first/usr/lib64/libQt5Quick.so.5.15.5,second/usr/lib64/libQt5Quick.so.5.15.5,missing-previously-found-exports,0.254755,symbols,True,True
2,fedora-libs-35,fedora-libs-37,first/usr/lib64/libQt5Quick.so.5.15.5,second/usr/lib64/libQt5Quick.so.5.15.5,abidiff,49.429475,libabigail,True,True
3,fedora-libs-35,fedora-libs-37,first/usr/lib64/libQt5Quick.so.5.15.5,second/usr/lib64/libQt5Quick.so.5.15.5,abi-compliance-tester,625.529167,abi-laboratory,True,True
4,fedora-libs-34,fedora-libs-37,first/usr/lib64/libx11globalcomm.so.1.0.0,second/usr/lib64/libx11globalcomm.so.1.0.0,missing-previously-found-symbols,0.000072,symbols,True,True
...,...,...,...,...,...,...,...,...,...
28675,fedora-libs-34,fedora-libs-36,first/usr/lib/systemd/libsystemd-shared-250.so,second/usr/lib/systemd/libsystemd-shared-250.so,abi-compliance-tester,31.900113,abi-laboratory,True,True
28676,fedora-libs-36,fedora-libs-37,first/usr/lib/systemd/libsystemd-shared-250.so,second/usr/lib/systemd/libsystemd-shared-250.so,missing-previously-found-symbols,0.000897,symbols,True,True
28677,fedora-libs-36,fedora-libs-37,first/usr/lib/systemd/libsystemd-shared-250.so,second/usr/lib/systemd/libsystemd-shared-250.so,missing-previously-found-exports,0.106127,symbols,True,True
28678,fedora-libs-36,fedora-libs-37,first/usr/lib/systemd/libsystemd-shared-250.so,second/usr/lib/systemd/libsystemd-shared-250.so,abidiff,1.399777,libabigail,True,True


In [6]:
# Convert to wide format where analysis and predictor are columns.
df_wide = pandas.pivot(df, columns=['analysis', 'predictor'], index=['a', 'b', 'original', 'changed'], values='prediction')
df_wide

analysis                                                                                                                            missing-previously-found-symbols  \
predictor                                                                                                                                                    symbols   
a              b              original                                           changed                                                                               
fedora-libs-34 fedora-libs-35 first/usr/lib/gcc/x86_64-redhat-linux/12/32/lib... second/usr/lib/gcc/x86_64-redhat-linux/12/32/li...                              NaN   
                              first/usr/lib/gcc/x86_64-redhat-linux/12/32/lib... second/usr/lib/gcc/x86_64-redhat-linux/12/32/li...                              NaN   
                              first/usr/lib/gcc/x86_64-redhat-linux/12/32/lib... second/usr/lib/gcc/x86_64-redhat-linux/12/32/li...                              NaN   
                              first/usr/lib/gcc/x86_64-redhat-linux/12/32/lib... second/usr/lib/gcc/x86_64-redhat-linux/12/32/li...                              NaN   
                              first/usr/lib/gcc/x86_64-redhat-linux/12/32/lib... second/usr/lib/gcc/x86_64-redhat-linux/12/32/li...                              NaN   
...                                                                                                                                                              ...   
fedora-libs-36 fedora-libs-37 first/usr/lib64/python3.10/lib-dynload/_decimal... second/usr/lib64/python3.10/lib-dynload/_decima...                             True   
                              first/usr/lib64/python3.10/lib-dynload/_element... second/usr/lib64/python3.10/lib-dynload/_elemen...                             True   
                              first/usr/lib64/python3.10/lib-dynload/_gdbm.cp... second/usr/lib64/python3.10/lib-dynload/_gdbm.c...                             True   
                              first/usr/lib64/python3.10/lib-dynload/_hashlib... second/usr/lib64/python3.10/lib-dynload/_hashli...                             True   
                              first/usr/lib64/python3.10/lib-dynload/_heapq.c... second/usr/lib64/python3.10/lib-dynload/_heapq....                             True   

analysis                                                                                                                            missing-previously-found-exports  \
predictor                                                                                                                                                    symbols   
a              b              original                                           changed                                                                               
fedora-libs-34 fedora-libs-35 first/usr/lib/gcc/x86_64-redhat-linux/12/32/lib... second/usr/lib/gcc/x86_64-redhat-linux/12/32/li...                              NaN   
                              first/usr/lib/gcc/x86_64-redhat-linux/12/32/lib... second/usr/lib/gcc/x86_64-redhat-linux/12/32/li...                              NaN   
                              first/usr/lib/gcc/x86_64-redhat-linux/12/32/lib... second/usr/lib/gcc/x86_64-redhat-linux/12/32/li...                              NaN   
                              first/usr/lib/gcc/x86_64-redhat-linux/12/32/lib... second/usr/lib/gcc/x86_64-redhat-linux/12/32/li...                              NaN   
                              first/usr/lib/gcc/x86_64-redhat-linux/12/32/lib... second/usr/lib/gcc/x86_64-redhat-linux/12/32/li...                              NaN   
...                                                                                                                                                              ...   
fedora-libs-36 fedora-libs-37 first/usr/lib64/python3.10/lib-dynload/_decimal... second/usr/lib64/python3.10/lib-dynload/_decima...    

In [7]:
# Join analysis/predictor columns
df_wide = df_wide.reset_index()
df_wide.columns = ["_".join(names) for names in df_wide.columns]
df_wide.columns

Index(['a_', 'b_', 'original_', 'changed_',
       'missing-previously-found-symbols_symbols',
       'missing-previously-found-exports_symbols', 'abidiff_libabigail',
       'abi-compliance-tester_abi-laboratory'],
      dtype='object')

In [8]:
# Get predictions only
df_predictions = df_wide[['missing-previously-found-symbols_symbols',
                          'missing-previously-found-exports_symbols',
                          'abidiff_libabigail',
                          'abi-compliance-tester_abi-laboratory']]
df_predictions

,missing-previously-found-symbols_symbols,missing-previously-found-exports_symbols,abidiff_libabigail,abi-compliance-tester_abi-laboratory
0,NaN,NaN,False,False
1,NaN,NaN,False,False
2,NaN,NaN,False,False
3,NaN,NaN,False,False
4,NaN,NaN,False,False
...,...,...,...,...
7207,True,True,True,True
7208,True,True,True,True
7209,True,True,True,True
7210,True,True,True,True


In [9]:
# Filter to where predictions are either "True" or "False" only.
num_predictors = 4
df_predictions = df_predictions[(df_predictions == True).sum(axis=1) + (df_predictions == False).sum(axis=1) == num_predictors]

# Look to see if we have True/False
for column in df_predictions.columns:
    print(column)
    print(df_predictions[column].unique())
    
df_predictions

missing-previously-found-symbols_symbols
[True]
missing-previously-found-exports_symbols
[True]
abidiff_libabigail
[True False]
abi-compliance-tester_abi-laboratory
[True False]


,missing-previously-found-symbols_symbols,missing-previously-found-exports_symbols,abidiff_libabigail,abi-compliance-tester_abi-laboratory
9,True,True,True,True
13,True,True,True,True
14,True,True,True,True
15,True,True,True,True
16,True,True,True,True
...,...,...,...,...
7207,True,True,True,True
7208,True,True,True,True
7209,True,True,True,True
7210,True,True,True,True


In [10]:
# Correlations between predictors.
# The NaN means that the values don't vary (as we see above, the symbols check is always true)
(df_predictions == True).corr()

,missing-previously-found-symbols_symbols,missing-previously-found-exports_symbols,abidiff_libabigail,abi-compliance-tester_abi-laboratory
missing-previously-found-symbols_symbols,NaN,NaN,NaN,NaN
missing-previously-found-exports_symbols,NaN,NaN,NaN,NaN
abidiff_libabigail,NaN,NaN,1.000000,0.388571
abi-compliance-tester_abi-laboratory,NaN,NaN,0.388571,1.000000
